In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from utils import Utils
from utils import k_fold_cross
from smote import Smote
from logistic_regression_2 import LogisticRegression

In [2]:
## Reading the data
df = pd.read_csv('students_adaptability_level_online_education.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Gender               1205 non-null   object
 1   Age                  1205 non-null   object
 2   Education Level      1205 non-null   object
 3   Institution Type     1205 non-null   object
 4   IT Student           1205 non-null   object
 5   Location             1205 non-null   object
 6   Load-shedding        1205 non-null   object
 7   Financial Condition  1205 non-null   object
 8   Internet Type        1205 non-null   object
 9   Network Type         1205 non-null   object
 10  Class Duration       1205 non-null   object
 11  Self Lms             1205 non-null   object
 12  Device               1205 non-null   object
 13  Adaptivity Level     1205 non-null   object
dtypes: object(14)
memory usage: 131.9+ KB


In [3]:
# Checking the features
df.describe().T

,count,unique,top,freq
Gender,1205,2,Boy,663
Age,1205,6,21-25,374
Education Level,1205,3,School,530
Institution Type,1205,2,Non Government,823
IT Student,1205,2,No,901
Location,1205,2,Yes,935
Load-shedding,1205,2,Low,1004
Financial Condition,1205,3,Mid,878
Internet Type,1205,2,Mobile Data,695
Network Type,1205,3,4G,775


In [4]:
preprocesser = Utils(df)

In [5]:
## Preprocessing the categorical data and applying one-hot-encoding to the response
df.iloc[:, :-1] = preprocesser.labeling()
df = df.drop(['Adaptivity Level'], axis=1)
one_hot_response = preprocesser.one_hot_encoding()
#df = one_hot_response
df['Adaptivity Level_High'] = one_hot_response['Adaptivity Level_High']
df['Adaptivity Level_Low'] = one_hot_response['Adaptivity Level_Low']
df['Adaptivity Level_Moderate'] = one_hot_response['Adaptivity Level_Moderate']

/Users/besteguney/Documents/fourth_year/spring/eee485/Project_Github/EEE485/utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_df[column] = self.data[column].map(mapping)
/Users/besteguney/Documents/fourth_year/spring/eee485/Project_Github/EEE485/utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_df[column] = self.data[column].map(mapping)
/Users/besteguney/Documents/fourth_year/spring/eee485/Project_Github/EEE485/utils.py:28: SettingWithCopyWarning: 
A value is trying to be 

In [6]:
df

,Gender,Age,Education Level,Institution Type,IT Student,Location,Load-shedding,Financial Condition,Internet Type,Network Type,Class Duration,Self Lms,Device,Adaptivity Level_High,Adaptivity Level_Low,Adaptivity Level_Moderate
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,1,0,1,0,1,1,1,0,0,1
2,1,1,1,1,0,0,0,0,0,0,1,0,1,0,0,1
3,1,2,2,0,0,0,0,0,1,0,1,0,1,0,0,1
4,1,1,2,0,0,0,0,1,1,1,2,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,1,1,1,0,0,0,0,0,0,0,1,0,1,0,1,0
1201,1,1,1,0,0,1,1,0,0,0,0,0,1,0,0,1
1202,0,2,2,0,0,0,0,0,1,1,1,0,1,0,0,1
1203,1,1,1,0,0,1,0,0,0,0,1,0,1,0,1,0


In [7]:
## Handling imbalances in data
smote = Smote(df, 100, 5)
generated_data = smote.smote()
generated_data = pd.DataFrame(generated_data, columns=df.iloc[:, :-3].columns)
generated_data['Adaptivity Level_Low'] = 0
generated_data['Adaptivity Level_High'] = 1
generated_data['Adaptivity Level_Moderate'] = 0

<class 'numpy.ndarray'>


In [8]:
df = df.append(generated_data)

/var/folders/xn/ckzmp8n52bj64l343d38bl4r0000gn/T/ipykernel_15460/3962297237.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(generated_data)


In [9]:
df.reset_index(drop=True, inplace=True)

In [10]:
# Checking the features
df.tail()

,Gender,Age,Education Level,Institution Type,IT Student,Location,Load-shedding,Financial Condition,Internet Type,Network Type,Class Duration,Self Lms,Device,Adaptivity Level_High,Adaptivity Level_Low,Adaptivity Level_Moderate
1300,0,3,0,1,1,0,0,2,0,0,1,0,1,1,0,0
1301,0,3,0,1,1,0,0,2,0,0,1,0,2,1,0,0
1302,1,2,2,0,0,0,0,2,0,1,1,0,1,1,0,0
1303,0,0,0,0,1,0,1,1,0,0,1,1,2,1,0,0
1304,0,0,0,0,1,0,1,1,0,0,1,1,2,1,0,0


In [ ]:
# Logistic Regression
logistic_regression = LogisticRegression(0.03, 10000)
print(k_fold_cross(logistic_regression, df, 10, 1))

/Users/besteguney/Documents/fourth_year/spring/eee485/Project_Github/EEE485/utils.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train1.append(train2, ignore_index=True)
/Users/besteguney/Documents/fourth_year/spring/eee485/Project_Github/EEE485/utils.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train1.append(train2, ignore_index=True)


In [12]:
# Logistic Regression
logistic_regression = LogisticRegression(0.03, 10000)
df_1 = df.drop(['Adaptivity Level_High', 'Adaptivity Level_Low'], axis=1)
xtrain = df_1.iloc[:-200, :-1]
ytrain = df_1.iloc[:-200, -1:]
weight = np.zeros((13,1))
parameters = logistic_regression.fit(xtrain, ytrain, weight)


In [12]:
parameters

,Adaptivity Level_Moderate
Gender,0.210461
Age,0.045341
Education Level,0.647982
Institution Type,-0.294632
IT Student,0.824562
Location,-0.189960
Load-shedding,-0.348099
Financial Condition,-0.720035
Internet Type,0.929396
Network Type,-0.745452


In [13]:
xtest = df_1.iloc[-200:, :-1]
ytest = df_1.iloc[-200:, -1:]
prediction1 = logistic_regression.predict(xtest, parameters)

In [14]:
df_2 = df.drop(['Adaptivity Level_Moderate', 'Adaptivity Level_Low'], axis=1)
xtrain2 = df_2.iloc[:-200, :-1]
ytrain2 = df_2.iloc[:-200, -1:]

weight = np.zeros((13,1))
parameters2 = logistic_regression.fit(xtrain2, ytrain2, weight)

df_3 = df.drop(['Adaptivity Level_High', 'Adaptivity Level_Moderate'], axis=1)
xtrain3 = df_3.iloc[:-200, :-1]
ytrain3 = df_3.iloc[:-200, -1:]
weight = np.zeros((13,1))
parameters3 = logistic_regression.fit(xtrain3, ytrain3, weight)

In [15]:
xtest = df_2.iloc[-200:, :-1]
ytest = df_2.iloc[-200:, -1:]
prediction2 = logistic_regression.predict(xtest, parameters2)

xtest = df_3.iloc[-200:, :-1]
ytest = df_3.iloc[-200:, -1:]
prediction3 = logistic_regression.predict(xtest, parameters3)

In [16]:
classified = logistic_regression.classify(prediction1, prediction2, prediction3)

In [18]:
classified.shape

(200, 3)

In [20]:
test_pred.head()

,Adaptivity Level_Moderate
27,1.463311
601,-2.338190
26,1.440982
1139,-2.613549
1173,0.154385


In [21]:
train = df_2.iloc[:-200, :]
test = df_2.iloc[-200:, :].astype(float)
lr_2 = LogisticRegression(train, 0.03, 10000)
test_pred_2 = lr_2.prediction(test.iloc[:, :-1])

In [22]:
train = df_3.iloc[:-200, :]
test = df_3.iloc[-200:, :].astype(float)
lr_3 = LogisticRegression(train, 0.03, 10000)
test_pred_3 = lr_3.prediction(test.iloc[:, :-1])

In [23]:
test_pred['Adaptivity Level_High'] = test_pred_2['Adaptivity Level_High']
test_pred['Adaptivity Level_Low'] = test_pred_3['Adaptivity Level_Low']

In [24]:
test_pred.head()

,Adaptivity Level_Moderate,Adaptivity Level_High,Adaptivity Level_Low
27,1.463311,-1.274215,-2.832706
601,-2.338190,-2.866957,1.633697
26,1.440982,-2.698367,-2.058776
1139,-2.613549,-2.555163,2.637632
1173,0.154385,-4.613115,-0.006119


In [31]:
replace_fn = lambda row: [1 if x == max(row) else 0 for x in row]

# apply the function to each row of the DataFrame
test_pred = test_pred.apply(replace_fn, axis=1, result_type='expand')
test_pred.columns = ['Adaptivity Level_Moderate','Adaptivity Level_High', 'Adaptivity Level_Low']

In [51]:
test_pred.head()
test_pred = test_pred.reindex(columns=['Adaptivity Level_High', 'Adaptivity Level_Low', 'Adaptivity Level_Moderate'])

In [45]:
df.iloc[-200:, -3:].head()

,Adaptivity Level_High,Adaptivity Level_Low,Adaptivity Level_Moderate
27,0,0,1
601,0,1,0
26,0,0,1
1139,0,1,0
1173,0,1,0


In [ ]:
orig = df.iloc[-200:, -3:].reset_index(drop=True)
for index in range(0,200):
    

In [49]:
# merge the data frames on the three columns
merged = pd.merge(df.iloc[-200:, -3:], test_pred, on=['Adaptivity Level_High', 'Adaptivity Level_Low', 'Adaptivity Level_Moderate'])

# filter out the rows where the values in the three columns are the same
filtered = merged[merged.duplicated(subset=['Adaptivity Level_High', 'Adaptivity Level_Low', 'Adaptivity Level_Moderate'], keep=False) == False]


In [50]:
filtered

,Adaptivity Level_High,Adaptivity Level_Low,Adaptivity Level_Moderate


In [41]:
df.iloc[-200:, :][df['Adaptivity Level_High'] == test_pred['Adaptivity Level_High']]

ValueError: Can only compare identically-labeled Series objects

In [52]:
df.iloc[-200:, -3:].compare(test_pred)

Adaptivity Level_High       Adaptivity Level_Low        \
                      self other                 self other   
1173                   NaN   NaN                  1.0   0.0   
294                    0.0   1.0                  1.0   0.0   
666                    NaN   NaN                  1.0   0.0   
54                     NaN   NaN                  1.0   0.0   
1051                   NaN   NaN                  0.0   1.0   
...                    ...   ...                  ...   ...   
135                    NaN   NaN                  1.0   0.0   
263                    NaN   NaN                  1.0   0.0   
844                    1.0   0.0                  NaN   NaN   
5                      NaN   NaN                  1.0   0.0   
233                    1.0   0.0                  NaN   NaN   

     Adaptivity Level_Moderate        
                          self other  
1173                       0.0   1.0  
294                        NaN   NaN  
666                        0.0   1.0  
54                         0.0   1.0  
1051                       1.0   0.0  
...                        ...   ...  
135                        0.0   1.0  
263                        0.0   1.0  
844                        0.0   1.0  
5                          0.0   1.0  
233                        0.0   1.0  

[61 rows x 6 columns]